In [1]:
import numpy as np
import keras
import os
from pathlib import Path

C:\Users\dmitr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
batch_size = 64  # Розмір пакету для навчання.
epochs = 100  # Кількість епох для навчання.
latent_dim = 256  # Розмірність прихованого простору кодування.
num_samples = 150000  # Кількість зразків для навчання.
# Шлях до текстового файлу даних на диску.
data_path = os.path.join("ukr.txt")

In [3]:
# Векторизуємо дані.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

# Відкриваємо файл з даними і читаємо його рядок за рядком.
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

# Розбиваємо кожен рядок на вхідний текст (input_text) та цільовий текст (target_text).
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")

    # Додаємо спеціальні символи "початку послідовності" та "кінця послідовності" до цільового тексту.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)

    # Оновлюємо набори символів.
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

# Сортуємо символи.
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

# Обчислюємо різні параметри даних.
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

# Виводимо статистику.
print("Кількість зразків:", len(input_texts))
print("Кількість унікальних вхідних токенів:", num_encoder_tokens)
print("Кількість унікальних вихідних токенів:", num_decoder_tokens)
print("Максимальна довжина послідовності для входів:", max_encoder_seq_length)
print("Максимальна довжина послідовності для виходів:", max_decoder_seq_length)

# Створюємо словники для перетворення символів у індекси та навпаки.
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

# Ініціалізуємо дані для кодера та декодера.
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype="float32",
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)

# Заповнюємо дані для кодера та декодера.
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data випереджає decoder_input_data на один крок часу
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data буде випереджати на один крок часу
            # і не буде включати початковий символ.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Кількість зразків: 150000
Кількість унікальних вхідних токенів: 84
Кількість унікальних вихідних токенів: 148
Максимальна довжина послідовності для входів: 44
Максимальна довжина послідовності для виходів: 77


In [4]:
# Визначаємо вхідну послідовність та обробляємо її.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# Ми відкидаємо `encoder_outputs` і зберігаємо лише стани.
encoder_states = [state_h, state_c]

# Налаштовуємо декодер, використовуючи `encoder_states` як початковий стан.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# Ми налаштовуємо наш декодер повертати повні вихідні послідовності,
# а також повертати внутрішні стани. Ми не використовуємо
# повернені стани в моделі навчання, але ми будемо використовувати їх при виведенні.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Визначаємо модель, яка перетворить
# `encoder_input_data` та `decoder_input_data` в `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [5]:
# Компілюємо модель. Використовуємо оптимізатор "rmsprop", функцію втрат "categorical_crossentropy" та метрику "accuracy".
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)

# Навчаємо модель. Вхідні дані для кодера та декодера (`encoder_input_data`, `decoder_input_data`) перетворюються в цільові дані декодера (`decoder_target_data`).
# Використовуємо розмір пакету, вказаний раніше (`batch_size`), та кількість епох (`epochs`).
# Також відводимо 20% вхідних даних для валідації (`validation_split=0.2`).
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

# Зберігаємо модель після навчання.
model.save("s2s_model.keras")



Epoch 1/100


1875/1875 [==============================] - 438s 231ms/step - loss: 0.7913 - accuracy: 0.7845 - val_loss: 0.9664 - val_accuracy: 0.7167
Epoch 2/100
1875/1875 [==============================] - 426s 227ms/step - loss: 0.5731 - accuracy: 0.8330 - val_loss: 0.8211 - val_accuracy: 0.7613
Epoch 3/100
1875/1875 [==============================] - 419s 223ms/step - loss: 0.5008 - accuracy: 0.8543 - val_loss: 0.7690 - val_accuracy: 0.7758
Epoch 4/100
1875/1875 [==============================] - 413s 221ms/step - loss: 0.4585 - accuracy: 0.8663 - val_loss: 0.7048 - val_accuracy: 0.7958
Epoch 5/100
1875/1875 [==============================] - 419s 223ms/step - loss: 0.4269 - accuracy: 0.8754 - val_loss: 0.6599 - val_accuracy: 0.8083
Epoch 6/100
1875/1875 [==============================] - 422s 225ms/step - loss: 0.4020 - accuracy: 0.8829 - val_loss: 0.6304 - val_accuracy: 0.8170
Epoch 7/100
1875/1875 [==============================] - 412s 220ms/step - loss: 0.3835 - accuracy: 0.8

In [6]:
# Визначаємо моделі вибірки
# Відновлюємо модель та створюємо кодер та декодер.
model = keras.models.load_model("s2s_model.keras")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Індекс токенів для декодування послідовностей назад до
# чогось зрозумілого.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Кодуємо вхід як вектори станів.
    states_value = encoder_model.predict(input_seq, verbose=0)

    # Генеруємо порожню цільову послідовність довжиною 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Заповнюємо перший символ цільової послідовності символом початку.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Цикл вибірки для пакету послідовностей
    # (для спрощення тут ми припускаємо пакет розміром 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )

        # Вибираємо токен
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Умова виходу: або досягнуто максимальної довжини
        # або знайдено символ зупинки.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Оновлюємо цільову послідовність (довжиною 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Оновлюємо стани
        states_value = [h, c]
    return decoded_sentence

In [8]:
# Речення для перекладу
my_sentence = "How about playing some games"

# Кодуємо наше речення так само, як ми це робили під час підготовки даних.
encoder_input_data = np.zeros(
    (1, max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
for t, char in enumerate(my_sentence):
    if char in input_token_index:
        encoder_input_data[0, t, input_token_index[char]] = 1.0
encoder_input_data[0, t + 1 :, input_token_index.get(" ", 0)] = 1.0


# Використовуємо модель для декодування речення
decoded_sentence = decode_sequence(encoder_input_data[0:1])

print("-")
print("Вхідне речення:", my_sentence)
print("Декодоване речення:", decoded_sentence)


-
Вхідне речення: How about playing some games
Декодоване речення: Як щодо твоє ім'я?



Як бачимо, якось він текст перекладає, але для навчання було взято лише 150000 семплів (11.5 годин). Тому для якісного перекладу необхідно більше часу та більший датасет.